In [1]:
from distutils.version import StrictVersion
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time
import imageio

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from tensorflow.compat.v1.keras import backend as K

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# TensorFlow wizardry
config = tf.compat.v1.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of all the GPU memory to be allocated
#config.gpu_options.per_process_gpu_memory_fraction = 1.0
 
# Create a session with the above options specified.
K.set_session(tf.compat.v1.Session(config=config))
#keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# This is needed to display the images.
%matplotlib inline

In [5]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [6]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = 'C:/Users/cdghub/Desktop/Beatriz/train/ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_smd_2_2019_01_29' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'C:/Users/cdghub/Desktop/Beatriz/train/label_map' + '/label_map.pbtxt'

NUM_CLASSES = 10

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v1.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [9]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
    (im_height, im_width, 3)).astype(np.uint8)

In [10]:
from tkinter import Tcl
PATH_TO_TEST_IMAGES_DIR = 'D:/Classes/test'
x = os.listdir(PATH_TO_TEST_IMAGES_DIR)
sort = Tcl().call('lsort', '-dict', x)
TEST_IMAGES = [os.path.join(PATH_TO_TEST_IMAGES_DIR ,img) for img in (sort)]

n = 0
# select only 50 for testing time!
TEST_IMAGES = TEST_IMAGES[0+n:5000]
print(len(TEST_IMAGES))

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
namessplit=[]

for i in range(0,len(TEST_IMAGES)):
    namessplit.append((sort[i+n].split('.'))[0])   
    
    

5000


In [11]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.compat.v1.Session() as sess:
        # Get handles to input and output tensors
            ops = tf.compat.v1.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
              'num_detections', 'detection_boxes', 'detection_scores',
              'detection_classes', 'detection_masks'
              ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.compat.v1.get_default_graph().get_tensor_by_name(
                    tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [12]:
import shutil
from os import listdir
from os.path import isfile, join, basename


def move(path_origem, path_destino, ext='zip'):
    for item in [join(path_origem, f) for f in listdir(path_origem) if isfile(join(path_origem, f)) and f.endswith(ext)]:
        #print(item)
        shutil.move(item, join(path_destino, basename(item)))
        print('moved "{}" -> "{}"'.format(item, join(path_destino, basename(item))))

In [13]:
# load images
images = []
start = time.time()
for i, image_path in enumerate(TEST_IMAGES):
    image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)    
#   # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    output_dict =run_inference_for_single_image(image_np, detection_graph)
    
    #images.append(image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
               image_np,
               output_dict['detection_boxes'],
               output_dict['detection_classes'],
               output_dict['detection_scores'],
               category_index,
               instance_masks=output_dict.get('detection_masks'),
               use_normalized_coordinates=True, #
               line_thickness=2)
    # This is the way I'm getting my coordinates
    #boxes = output_dict['detection_boxes']
    # get all boxes from an array
    #max_boxes_to_draw = boxes.shape[0]
    # get scores to get a threshold
    #scores = output_dict['detection_scores']
    # this is set as a default but feel free to adjust it to your needs
    #min_score_thresh=.4
    # generate txt files with object coordinates in the image and classification
    #f= open("teste/{}.txt".format((namessplit[i])),"w+")
    #List = []
    #coord = []
    #for j in range(min(max_boxes_to_draw, boxes.shape[0])):
        # 
     #   if scores[j] > min_score_thresh:
      #      #min_score_thresh = scores[j]
       #     # boxes[i] is the box which will be drawn
         #   class_name = category_index[output_dict['detection_classes'][j]]['name']
            #This is the number of the class
           # f.write("5 ")
            #for x in (boxes[j]):
                #List.append(x)

            #coordinates transformation
            #W = List[3] - List[1]
            #H = List[2] - List[0]
            #x1 = List[1] + (W/2)
            #y1 = List[2] - (H/2)
            #coord = [x1, y1, W, H]
            
            #for y in (coord):

                #f.write(str(y) + " ")
                
            #f.write(str(scores[j]))
            #f.write("\n") 
            #List.clear()
        
    #f.close() 
    #filesize = os.path.getsize("teste/{}.txt".format((namessplit[i])))
    #This exclude images with no object found (txt size = 0)
    #if filesize == 0:
        #shutil.move(("teste/{}.txt".format((namessplit[i]))),("teste/del/{}.txt".format((namessplit[i]))))
        #shutil.move(("D:/Classes/Fishing/Fishing/{}.jpg"((namessplit[i]))),("faster_rcnn_inception_v2_smd/del/{}.jpg".format((namessplit[i]))))
    #else:        
        #plt.figure(figsize=IMAGE_SIZE)
        #plt.imshow(image_np)
        #plt.savefig("teste/{}.png".format((namessplit[i])))
        

    i=i+1

print("loaded images")
end = time.time()
average_time_per_image = (end - start) / len(TEST_IMAGES)
print("Average time per image: ", average_time_per_image)
print("Frames per second: ", 1 / average_time_per_image)
# correct times. First image is warming up so it is not counting
print("Total time spent for " + str(len(TEST_IMAGES))+" images: ", (end - start)/60)

loaded images
Average time per image:  2.6390647360801696
Frames per second:  0.3789221182521314
Total time spent for 5000 images:  219.92206134001415
